In [1]:
import pandas as pd

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [47]:
rating = pd.read_csv('ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [41]:
rating_count = rating.groupby('userId').agg({'timestamp':['min', 'max', 'count']}).reset_index()
rating_count_100 = rating_count.loc[rating_count['timestamp']['count'] > 100]
rating_count_100['time life'] = rating_count_100['timestamp']['max'] - rating_count_100['timestamp']['min']
rating_count_100.head()

C:\Users\belle\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


userId   timestamp                   time life
                 min         max count          
0      1   964980499   965719662   232    739163
3      4   945078428  1007574542   216  62496114
5      6   845553109   845556915   314      3806
6      7  1106635416  1176181731   152  69546315
9     10  1455301553  1455619275   140    317722

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [110]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [111]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [112]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [113]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [132]:
client_revenue = (rzd
.merge(auto.set_index('client_id'), how='outer', on='client_id')
.merge(air.set_index('client_id'), how='outer', on='client_id')
)

client_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [134]:
client_address = (client_revenue
                  .merge(client_base.set_index('client_id'), how='outer', on='client_id'))

client_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3

> В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
 >> У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
 
 >> Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
 
 >> Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

___Для добавления координат в таблицы визитов и покупок нам нужны соответствующие идентификаторы визитов и покупок.
`visit_id`, `purchase_id`, чтобы их получить нужно ввести счетчики. Счетчик `visit_id` будет увеличиваться тогда когда пользователь будет заходить в приложение или сайт, а счетчик `purchase_id` будет увеличиться после совершения транзакции.
Дополнительные признаки которые мы можем получить из координат:___

_1) Мы можем найти места для каждого пользователя где он совершает больше всего покупок в радиусе условно 5км и как только он заходит в приложение или сайт, то отправляем ему рекламу или e-mail._

_2) В тех местах опять же условно в радиусе 5км, где пользователь покупает меньше всего продукции, можем кидать уникальное предложение(скидку) на самый покупаемый им продукт._

_3) Исходя из места нахождения (если это другая страна) можно предлагать те продукты которые производятся этой страной или ассоциируются с ней._